In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime,timedelta
import yfinance as yf
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import scipy.optimize
import mysql.connector
import pytz

c:\Users\igorb\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [15]:
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="bolsa_valores"
)
cursor = conn.cursor()


x

C:\Users\igorb\AppData\Local\Temp\ipykernel_6448\949717625.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  x = pd.read_sql("SELECT DISTINCT SYMBOL FROM cotacao", conn)


,SYMBOL
0,ABEV3
1,ALPA4
2,ALSO3
3,ARZZ3
4,AZUL4
...,...
94,RECV3
95,CBAV3
96,RAIZ4
97,IGTI1


In [41]:

x = pd.read_sql("SELECT DISTINCT SYMBOL FROM cotacao", conn)
symbol_list_ibrx = list(pd.read_csv('ibrx.csv',delimiter=";").index)
symbol_list_ibrx = symbol_list_ibrx + x['SYMBOL'].values.tolist()
symbol_list_ibrx = set(symbol_list_ibrx)
symbol_list_ibrx
df_final = pd.DataFrame()
for ativo in symbol_list_ibrx:
    if(ativo in x):
        chamada_api = yf.Ticker("PETR4.SA").history(period='1wk')
    else:    
        chamada_api = yf.Ticker("PETR4.SA").history(start = date(2020, 3, 23))
    df_hist = pd.DataFrame(chamada_api).reset_index()
    df_hist["Symbol"] = ativo
    df_hist = df_hist[["Date","Symbol","Open","High","Low","Close","Volume","Dividends"]]
    df_hist["Date"] = df_hist["Date"].dt.tz_convert(pytz.utc).dt.date
    df_hist["Date"] = df_hist["Date"].astype(str)
    df_final = pd.concat([df_hist, df_final],axis=0)

tuplas = tuple(df_final.itertuples(index=False, name=None))
query = "INSERT IGNORE INTO cotacao  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(query, tuplas)
conn.commit()
conn.close()

C:\Users\igorb\AppData\Local\Temp\ipykernel_6448\3386159386.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  x = pd.read_sql("SELECT DISTINCT SYMBOL FROM cotacao", conn)


In [26]:
y = pd.read_sql("SELECT DATE FROM cotacao", conn)


C:\Users\igorb\AppData\Local\Temp\ipykernel_6448\3286030288.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  y = pd.read_sql("SELECT DATE FROM cotacao", conn)


In [40]:
y = y.max()
y = date(y.year,y.month,y.day)
chamada_api = yf.Ticker("PETR4.SA").history(start = y)

TypeError: 'datetime.date' object is not callable

In [38]:
date(y)

TypeError: an integer is required (got type datetime.date)

In [36]:
chamada_api = yf.Ticker("PETR4.SA").history(start = date(2020, 3, 23))
date(2020, 3, 23)

datetime.date(2020, 3, 23)